# Quality Control

### Requirements
- seqkit
- seqtk 

In [6]:
import os, sys
from pathlib import Path

import pandas as pd
import numpy as np

In [3]:
# import utils.py
currentdir = os.path.dirname(os.path.realpath(__name__))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from utils import *

In [4]:
config_file = "config.toml"
config = parse_config(config_file)
root_dir = Path(config['root_dir'])

### Input

In [108]:
raw_fpath_file = root_dir / 'raw_fpath.csv'
raw_fpath_file

PosixPath('data/raw_fpath.csv')

### Onput

In [110]:
trimmed_fpath_file = root_dir / 'trimmed_fpath.csv'
trimmed_fpath_file

PosixPath('data/trimmed_fpath.csv')

## 1. Load file path

In [114]:
fpath_df = pd.read_csv(raw_fpath_file)
fpath_df

,running_id,raw_r1,raw_r2
0,SRR6046075,data/raw/SRR6046075_1.fastq.gz,data/raw/SRR6046075_2.fastq.gz
1,SRR6046701,data/raw/SRR6046701_1.fastq.gz,data/raw/SRR6046701_2.fastq.gz
2,SRR6045735,data/raw/SRR6045735_1.fastq.gz,data/raw/SRR6045735_2.fastq.gz
3,SRR6045327,data/raw/SRR6045327_1.fastq.gz,data/raw/SRR6045327_2.fastq.gz
4,SRR6045549,data/raw/SRR6045549_1.fastq.gz,data/raw/SRR6045549_2.fastq.gz
5,SRR6046105,data/raw/SRR6046105_1.fastq.gz,data/raw/SRR6046105_2.fastq.gz
6,SRR6046861,data/raw/SRR6046861_1.fastq.gz,data/raw/SRR6046861_2.fastq.gz
7,SRR6044910,data/raw/SRR6044910_1.fastq.gz,data/raw/SRR6044910_2.fastq.gz
8,SRR6045763,data/raw/SRR6045763_1.fastq.gz,data/raw/SRR6045763_2.fastq.gz
9,SRR6046306,data/raw/SRR6046306_1.fastq.gz,data/raw/SRR6046306_2.fastq.gz


## 2. Encode stat

### Bases

In [72]:
def get_seqkit_stat(fastq):
    cmd = f"seqkit stat {fastq}"
    output = subprocs(cmd)
    data = [line.split() for line in output.stdout.decode().strip().split('\n')]
    if len(data[1]) == 6:
        v = [data[1][0], data[1][1], data[1][2], 0, 0, 0, 0.0, 0]
        data[1] = v
    stat_ser = pd.Series(data[1], index=data[0])
    for col in ['num_seqs', 'sum_len', 'min_len', 'max_len']:
        stat_ser[col] = int(stat_ser[col].replace(',',''))
    for col in ['avg_len']:
        stat_ser[col] = float(stat_ser[col].replace(',',''))
    return stat_ser

def get_paired_fastq_stat(r1_fastq, r2_fastq):
    ser1 = get_seqkit_stat(r1_fastq)
    ser2 = get_seqkit_stat(r2_fastq)
    
    num_seqs = ser1['num_seqs'] + ser2['num_seqs']
    sum_len = ser1['sum_len'] + ser2['sum_len']
    
    return num_seqs, sum_len, sum_len / num_seqs

In [84]:
out, elp = execute_function_pool_args(
    get_paired_fastq_stat,
    fpath_df[['raw_r1', 'raw_r2']].values,
    4,
)

In [85]:
elp

21.427603244781494

In [87]:
base_stat_df = pd.DataFrame(out, columns = ['num_seqs', 'sum_len', 'avg_len'])
base_stat_df

,num_seqs,sum_len,avg_len
0,2150140,315603941,146.782973
1,2181260,329370260,151.000000
2,2730788,412348988,151.000000
3,2368378,348449021,147.125594
4,2200232,312069380,141.834761
5,2739142,397357271,145.066328
6,2046324,300436707,146.817761
7,1964346,275329828,140.163611
8,2311282,344349323,148.986287
9,2545170,339698387,133.467858


In [88]:
!seqkit stat data/raw/SRR6046075_1.fastq.gz data/raw/SRR6046075_2.fastq.gz

file                            format  type   num_seqs      sum_len  min_len  avg_len  max_len
data/raw/SRR6046075_1.fastq.gz  FASTQ   DNA   1,075,070  157,706,915        5    146.7      151
data/raw/SRR6046075_2.fastq.gz  FASTQ   DNA   1,075,070  157,897,026        5    146.9      151


In [89]:
!seqkit stat -a data/raw/SRR6046075_1.fastq.gz data/raw/SRR6046075_2.fastq.gz

file                            format  type   num_seqs      sum_len  min_len  avg_len  max_len   Q1   Q2   Q3  sum_gap  N50  Q20(%)  Q30(%)  GC(%)
data/raw/SRR6046075_1.fastq.gz  FASTQ   DNA   1,075,070  157,706,915        5    146.7      151  150  151  151        0  151    95.5   93.41  64.84
data/raw/SRR6046075_2.fastq.gz  FASTQ   DNA   1,075,070  157,897,026        5    146.9      151  150  151  151        0  151   85.69   80.08  65.27


### Error rate

In [90]:
from Bio import SeqIO
import math
import gzip

def cal_error_rate(r1_fastq, r2_fastq):
    phred_qual_counts = [0 for _ in range(43)]
    for fastq in (r1_fastq, r2_fastq):
        with gzip.open(fastq, 'rt') as hdl:
            for record in SeqIO.parse(hdl, "fastq"):
                for q in record.letter_annotations['phred_quality']:
                    phred_qual_counts[q] += 1
    
    err_count = 0
    for idx, count in enumerate(phred_qual_counts):
        err_count += math.pow(10, idx/(-10)) * count
    
    return err_count / sum(phred_qual_counts)

In [91]:
out, elp = execute_function_pool_args(
    cal_error_rate,
    fpath_df[['raw_r1', 'raw_r2']].values,
    4,
)

In [92]:
elp

238.63122296333313

In [95]:
error_rates = pd.Series(out)
error_rates.name = 'error_rate'
error_rates

0    0.004489
1    0.002960
2    0.004648
3    0.003844
4    0.004359
5    0.007210
6    0.005184
7    0.002015
8    0.011394
9    0.004654
Name: error_rate, dtype: float64

## 3. FastQC

showtime

## 4. Trimming

In [96]:
trimmed_dir = root_dir / 'trimmed'
trimmed_dir.mkdir(exist_ok=True)
trimmed_dir

PosixPath('data/trimmed')

In [97]:
def run_trimming(r1_fastq, r2_fastq):
    r1_fastq = Path(r1_fastq)
    r2_fastq = Path(r2_fastq)
    
    trimmed_r1 = trimmed_dir / (r1_fastq.name.split('.')[0]+'_trimmed.fastq.gz')
    unpaired_r1 = trimmed_dir / (r1_fastq.name.split('.')[0]+'_unpaired.fastq.gz')
    trimmed_r2 = trimmed_dir / (r2_fastq.name.split('.')[0]+'_trimmed.fastq.gz')
    unpaired_r2 = trimmed_dir / (r2_fastq.name.split('.')[0]+'_unpaired.fastq.gz')
    log_file = trimmed_dir / (r1_fastq.name.split('_1')[0]+'.log')
    
    if not (
        trimmed_r1.exists() &
        unpaired_r1.exists() &
        trimmed_r2.exists() &
        unpaired_r2.exists() &
        log_file.exists()
    ):
        cmd = f"trimmomatic PE {r1_fastq} {r2_fastq} \
            {trimmed_r1} {unpaired_r1} \
            {trimmed_r2} {unpaired_r2} \
            TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:25 -phred33 2> {log_file}"
        subprocs(cmd)
    
    return trimmed_r1, unpaired_r1, trimmed_r2, unpaired_r2, log_file

In [98]:
out, elp = execute_function_pool_args(
    run_trimming,
    fpath_df[['raw_r1', 'raw_r2']].values,
    4,
)

In [99]:
# elp
354.85028409957886

354.85028409957886

In [100]:
trimmed_fpath_df = pd.DataFrame(out, columns=['trimmed_r1', 'unpaired_r1', 'trimmed_r2', 'unpaired_r2', 'trimmomatic_log'])
trimmed_fpath_df

,trimmed_r1,unpaired_r1,trimmed_r2,unpaired_r2,trimmomatic_log
0,data/trimmed/SRR6046075_1_trimmed.fastq.gz,data/trimmed/SRR6046075_1_unpaired.fastq.gz,data/trimmed/SRR6046075_2_trimmed.fastq.gz,data/trimmed/SRR6046075_2_unpaired.fastq.gz,data/trimmed/SRR6046075.log
1,data/trimmed/SRR6046701_1_trimmed.fastq.gz,data/trimmed/SRR6046701_1_unpaired.fastq.gz,data/trimmed/SRR6046701_2_trimmed.fastq.gz,data/trimmed/SRR6046701_2_unpaired.fastq.gz,data/trimmed/SRR6046701.log
2,data/trimmed/SRR6045735_1_trimmed.fastq.gz,data/trimmed/SRR6045735_1_unpaired.fastq.gz,data/trimmed/SRR6045735_2_trimmed.fastq.gz,data/trimmed/SRR6045735_2_unpaired.fastq.gz,data/trimmed/SRR6045735.log
3,data/trimmed/SRR6045327_1_trimmed.fastq.gz,data/trimmed/SRR6045327_1_unpaired.fastq.gz,data/trimmed/SRR6045327_2_trimmed.fastq.gz,data/trimmed/SRR6045327_2_unpaired.fastq.gz,data/trimmed/SRR6045327.log
4,data/trimmed/SRR6045549_1_trimmed.fastq.gz,data/trimmed/SRR6045549_1_unpaired.fastq.gz,data/trimmed/SRR6045549_2_trimmed.fastq.gz,data/trimmed/SRR6045549_2_unpaired.fastq.gz,data/trimmed/SRR6045549.log
5,data/trimmed/SRR6046105_1_trimmed.fastq.gz,data/trimmed/SRR6046105_1_unpaired.fastq.gz,data/trimmed/SRR6046105_2_trimmed.fastq.gz,data/trimmed/SRR6046105_2_unpaired.fastq.gz,data/trimmed/SRR6046105.log
6,data/trimmed/SRR6046861_1_trimmed.fastq.gz,data/trimmed/SRR6046861_1_unpaired.fastq.gz,data/trimmed/SRR6046861_2_trimmed.fastq.gz,data/trimmed/SRR6046861_2_unpaired.fastq.gz,data/trimmed/SRR6046861.log
7,data/trimmed/SRR6044910_1_trimmed.fastq.gz,data/trimmed/SRR6044910_1_unpaired.fastq.gz,data/trimmed/SRR6044910_2_trimmed.fastq.gz,data/trimmed/SRR6044910_2_unpaired.fastq.gz,data/trimmed/SRR6044910.log
8,data/trimmed/SRR6045763_1_trimmed.fastq.gz,data/trimmed/SRR6045763_1_unpaired.fastq.gz,data/trimmed/SRR6045763_2_trimmed.fastq.gz,data/trimmed/SRR6045763_2_unpaired.fastq.gz,data/trimmed/SRR6045763.log
9,data/trimmed/SRR6046306_1_trimmed.fastq.gz,data/trimmed/SRR6046306_1_unpaired.fastq.gz,data/trimmed/SRR6046306_2_trimmed.fastq.gz,data/trimmed/SRR6046306_2_unpaired.fastq.gz,data/trimmed/SRR6046306.log


## 5. Merge and save

In [115]:
new_fpath_df = pd.concat([
    fpath_df,
    base_stat_df,
    error_rates,
    trimmed_fpath_df,
], axis=1)
new_fpath_df

,running_id,raw_r1,raw_r2,num_seqs,sum_len,avg_len,error_rate,trimmed_r1,unpaired_r1,trimmed_r2,unpaired_r2,trimmomatic_log
0,SRR6046075,data/raw/SRR6046075_1.fastq.gz,data/raw/SRR6046075_2.fastq.gz,2150140,315603941,146.782973,0.004489,data/trimmed/SRR6046075_1_trimmed.fastq.gz,data/trimmed/SRR6046075_1_unpaired.fastq.gz,data/trimmed/SRR6046075_2_trimmed.fastq.gz,data/trimmed/SRR6046075_2_unpaired.fastq.gz,data/trimmed/SRR6046075.log
1,SRR6046701,data/raw/SRR6046701_1.fastq.gz,data/raw/SRR6046701_2.fastq.gz,2181260,329370260,151.000000,0.002960,data/trimmed/SRR6046701_1_trimmed.fastq.gz,data/trimmed/SRR6046701_1_unpaired.fastq.gz,data/trimmed/SRR6046701_2_trimmed.fastq.gz,data/trimmed/SRR6046701_2_unpaired.fastq.gz,data/trimmed/SRR6046701.log
2,SRR6045735,data/raw/SRR6045735_1.fastq.gz,data/raw/SRR6045735_2.fastq.gz,2730788,412348988,151.000000,0.004648,data/trimmed/SRR6045735_1_trimmed.fastq.gz,data/trimmed/SRR6045735_1_unpaired.fastq.gz,data/trimmed/SRR6045735_2_trimmed.fastq.gz,data/trimmed/SRR6045735_2_unpaired.fastq.gz,data/trimmed/SRR6045735.log
3,SRR6045327,data/raw/SRR6045327_1.fastq.gz,data/raw/SRR6045327_2.fastq.gz,2368378,348449021,147.125594,0.003844,data/trimmed/SRR6045327_1_trimmed.fastq.gz,data/trimmed/SRR6045327_1_unpaired.fastq.gz,data/trimmed/SRR6045327_2_trimmed.fastq.gz,data/trimmed/SRR6045327_2_unpaired.fastq.gz,data/trimmed/SRR6045327.log
4,SRR6045549,data/raw/SRR6045549_1.fastq.gz,data/raw/SRR6045549_2.fastq.gz,2200232,312069380,141.834761,0.004359,data/trimmed/SRR6045549_1_trimmed.fastq.gz,data/trimmed/SRR6045549_1_unpaired.fastq.gz,data/trimmed/SRR6045549_2_trimmed.fastq.gz,data/trimmed/SRR6045549_2_unpaired.fastq.gz,data/trimmed/SRR6045549.log
5,SRR6046105,data/raw/SRR6046105_1.fastq.gz,data/raw/SRR6046105_2.fastq.gz,2739142,397357271,145.066328,0.007210,data/trimmed/SRR6046105_1_trimmed.fastq.gz,data/trimmed/SRR6046105_1_unpaired.fastq.gz,data/trimmed/SRR6046105_2_trimmed.fastq.gz,data/trimmed/SRR6046105_2_unpaired.fastq.gz,data/trimmed/SRR6046105.log
6,SRR6046861,data/raw/SRR6046861_1.fastq.gz,data/raw/SRR6046861_2.fastq.gz,2046324,300436707,146.817761,0.005184,data/trimmed/SRR6046861_1_trimmed.fastq.gz,data/trimmed/SRR6046861_1_unpaired.fastq.gz,data/trimmed/SRR6046861_2_trimmed.fastq.gz,data/trimmed/SRR6046861_2_unpaired.fastq.gz,data/trimmed/SRR6046861.log
7,SRR6044910,data/raw/SRR6044910_1.fastq.gz,data/raw/SRR6044910_2.fastq.gz,1964346,275329828,140.163611,0.002015,data/trimmed/SRR6044910_1_trimmed.fastq.gz,data/trimmed/SRR6044910_1_unpaired.fastq.gz,data/trimmed/SRR6044910_2_trimmed.fastq.gz,data/trimmed/SRR6044910_2_unpaired.fastq.gz,data/trimmed/SRR6044910.log
8,SRR6045763,data/raw/SRR6045763_1.fastq.gz,data/raw/SRR6045763_2.fastq.gz,2311282,344349323,148.986287,0.011394,data/trimmed/SRR6045763_1_trimmed.fastq.gz,data/trimmed/SRR6045763_1_unpaired.fastq.gz,data/trimmed/SRR6045763_2_trimmed.fastq.gz,data/trimmed/SRR6045763_2_unpaired.fastq.gz,data/trimmed/SRR6045763.log
9,SRR6046306,data/raw/SRR6046306_1.fastq.gz,data/raw/SRR6046306_2.fastq.gz,2545170,339698387,133.467858,0.004654,data/trimmed/SRR6046306_1_trimmed.fastq.gz,data/trimmed/SRR6046306_1_unpaired.fastq.gz,data/trimmed/SRR6046306_2_trimmed.fastq.gz,data/trimmed/SRR6046306_2_unpaired.fastq.gz,data/trimmed/SRR6046306.log


In [116]:
new_fpath_df.to_csv(trimmed_fpath_file, index=False)

In [37]:
cmd = f"vsearch --fastq_chars {fpath_df['r1'][0]}"
cmd

'vsearch --fastq_chars data/raw/SRR6046075_1.fastq.gz'

In [39]:
out = subprocs(cmd)

In [40]:
out

CompletedProcess(args='vsearch --fastq_chars data/raw/SRR6046075_1.fastq.gz', returncode=127, stdout=b'', stderr=b'/bin/sh: 1: vsearch: not found\n')

In [43]:
print(out.stderr.decode())

vsearch v2.21.1_linux_x86_64, 251.8GB RAM, 56 cores
https://github.com/torognes/vsearch

Reading FASTQ file 100%
Read 67766 sequences.
Qmin 33, QMax 73, Range 41
Guess: -fastq_qmin 0 -fastq_qmax 40 -fastq_ascii 33
Guess: Original Sanger format (phred+33)

Letter          N   Freq MaxRun
------ ---------- ------ ------
     A    1728398  23.2%     48
     C    1964634  26.4%     17
     G    1903135  25.6%     26
     N     103288   1.4%     34  Q=!
     T    1749014  23.5%     60

Char  ASCII    Freq       Tails
----  -----  ------  ----------
 '!'     33    1.4%        2963
 '-'     45    0.8%         166
 '.'     46    0.5%          48
 '/'     47    1.7%          94
 '0'     48    1.2%          52
 '1'     49    2.3%          61
 '2'     50    0.9%          24
 '3'     51    0.8%           6
 '4'     52    0.3%           6
 '5'     53    0.4%           1
 '6'     54    0.0%           0
 '9'     57    0.3%           1
 ':'     58    0.1%           0
 ';'     59    0.3%           0
 '